### Creating folder that contains super-resolution images for classification

In [ ]:
import os
import zipfile
import nibabel as nib
from PIL import Image
import numpy as np
import shutil

#Creating a new sub-directory to store files for the classification task
source_folder = r"D:\images"

HGG = os.listdir(r"D:\Gaussian_blur_bicubic_interpolation\HGG_HR\t1")
LGG = os.listdir(r"D:\Gaussian_blur_bicubic_interpolation\LGG_HR\t1")

if not os.path.exists(r"D:\Gaussian_blur_bicubic_interpolation\Classification\SR\HGG"):
    os.makedirs(r"D:\Gaussian_blur_bicubic_interpolation\Classification\SR\HGG")

if not os.path.exists(r"D:\Gaussian_blur_bicubic_interpolation\Classification\SR\LGG"):
    os.makedirs(r"D:\Gaussian_blur_bicubic_interpolation\Classification\SR\LGG")

#Change to how many epochs you trained the SwinIR model for
epoch_filter = 24

#Moving the files to the SR folder
for filename in os.listdir(source_folder):
    source_path = os.path.join(source_folder, filename, filename+"_"+str(epoch_filter) + ".png")
    if os.path.isfile(source_path):
        if filename + ".png" in HGG:
            shutil.copy(source_path, r"D:\Gaussian_blur_bicubic_interpolation\Classification\SR\HGG")
            os.rename(r"D:\Gaussian_blur_bicubic_interpolation\Classification\SR\HGG\\" + filename+"_"+str(epoch_filter) + ".png", r"D:\Gaussian_blur_bicubic_interpolation\Classification\SR\HGG\\" + filename + ".png")
        if filename + ".png" in LGG:
            shutil.copy(source_path, r"D:\Gaussian_blur_bicubic_interpolation\Classification\SR\LGG")
            os.rename(r"D:\Gaussian_blur_bicubic_interpolation\Classification\SR\LGG\\" + filename+"_"+str(epoch_filter) + ".png", r"D:\Gaussian_blur_bicubic_interpolation\Classification\SR\LGG\\" + filename + ".png")

### Creating Train-Test-Validation Splits for Classificaiton  (Stratified  Sampling)

#### SR

In [ ]:
import os
from sklearn.model_selection import train_test_split

HGG_LGG = ["HGG", "LGG"]
even_split = len(os.listdir(r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\LGG'))

#5 Different train-test-validation splits are created for cross-validation
for i in range(1,6):
    for glioma in HGG_LGG:
        data_folder = r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR' + "\\" + glioma
        all_files = os.listdir(data_folder)
        all_files = all_files[:even_split]
        train_files, test_and_val_files = train_test_split(all_files, test_size=0.2, random_state=i, shuffle=True)
        test_files, val_files = train_test_split(test_and_val_files, test_size=0.5, random_state=i, shuffle=True)

        # Define paths for train, test, and validation folders
        train_folder = r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\train\\'.format(i) + glioma
        test_folder = r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\test\\'.format(i) + glioma
        val_folder = r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\val\\'.format(i)  + glioma

        # Create train, test, and validation folders if they don't exist
        os.makedirs(train_folder, exist_ok=True)
        os.makedirs(test_folder, exist_ok=True)
        os.makedirs(val_folder, exist_ok=True)

            # Move files to their respective folders
        for file in train_files:
            src_path = os.path.join(data_folder, file)
            dest_path = os.path.join(train_folder, file)
            shutil.copy(src_path, dest_path)

        for file in test_files:
            src_path = os.path.join(data_folder, file)
            dest_path = os.path.join(test_folder, file)
            shutil.copy(src_path, dest_path)

        for file in val_files:
            src_path = os.path.join(data_folder, file)
            dest_path = os.path.join(val_folder, file)
            shutil.copy(src_path, dest_path)

#### HR

In [ ]:
HGG_LGG = ["HGG", "LGG"]

#5 Different train-test-validation splits are created for cross-validation
for i in range(1,6):
    for glioma in HGG_LGG:
        # Define paths for train, test, and validation folders
        train_folder = r'D:\Gaussian_blur_bicubic_interpolation\Classification\HR\{}\train\\'.format(i) + glioma
        test_folder = r'D:\Gaussian_blur_bicubic_interpolation\Classification\HR\{}\test\\'.format(i) + glioma
        val_folder = r'D:\Gaussian_blur_bicubic_interpolation\Classification\HR\{}\val\\'.format(i)  + glioma

        # Create train, test, and validation folders if they don't exist
        os.makedirs(train_folder, exist_ok=True)
        os.makedirs(test_folder, exist_ok=True)
        os.makedirs(val_folder, exist_ok=True)    

        train = r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\train\\'.format(i) + glioma
        test =  r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\test\\'.format(i) + glioma
        val =  r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\val\\'.format(i) + glioma

        #Copying the images in the train, test and val folder, so that they are the same for LR, HR, SR

        for file in os.listdir(train):
            if file in os.listdir(r'D:\Gaussian_blur_bicubic_interpolation\{}_HR\t1'.format(glioma)):
                shutil.copy(r'D:\Gaussian_blur_bicubic_interpolation\{}_HR\t1'.format(glioma)+"\\"+file, r"D:\Gaussian_blur_bicubic_interpolation\Classification\HR\{}\train\{}".format(i,glioma))

        for file in os.listdir(test):
            if file in os.listdir(r'D:\Gaussian_blur_bicubic_interpolation\{}_HR\t1'.format(glioma)):
                shutil.copy(r'D:\Gaussian_blur_bicubic_interpolation\{}_HR\t1'.format(glioma)+"\\"+file, r"D:\Gaussian_blur_bicubic_interpolation\Classification\HR\{}\test\{}".format(i,glioma))

        for file in os.listdir(val):
            if file in os.listdir(r'D:\Gaussian_blur_bicubic_interpolation\{}_HR\t1'.format(glioma)):
                shutil.copy(r'D:\Gaussian_blur_bicubic_interpolation\{}_HR\t1'.format(glioma)+"\\"+file, r"D:\Gaussian_blur_bicubic_interpolation\Classification\HR\{}\val\{}".format(i,glioma))

##### LR

In [ ]:
HGG_LGG = ["HGG", "LGG"]

#5 Different train-test-validation splits are created for cross-validation
for i in range(1,6):
    for glioma in HGG_LGG:
        # Define paths for train, test, and validation folders
        train_folder = r'D:\Gaussian_blur_bicubic_interpolation\Classification\LR\{}\train\\'.format(i) + glioma
        test_folder = r'D:\Gaussian_blur_bicubic_interpolation\Classification\LR\{}\test\\'.format(i) + glioma
        val_folder = r'D:\Gaussian_blur_bicubic_interpolation\Classification\LR\{}\val\\'.format(i)  + glioma

        # Create train, test, and validation folders if they don't exist
        os.makedirs(train_folder, exist_ok=True)
        os.makedirs(test_folder, exist_ok=True)
        os.makedirs(val_folder, exist_ok=True)    

        train = r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\train\\'.format(i) + glioma
        test =  r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\test\\'.format(i) + glioma
        val =  r'D:\Gaussian_blur_bicubic_interpolation\Classification\SR\{}\val\\'.format(i) + glioma

        #Copying the images in the train, test and val folder, so that they are the same for LR, HR, SR

        for file in os.listdir(train):
            if file in os.listdir(r'D:\Gaussian_blur_bicubic_interpolation\{}_LR\t1\4x'.format(glioma)):
                shutil.copy(r'D:\Gaussian_blur_bicubic_interpolation\{}_LR\t1\4x'.format(glioma)+"\\"+file, r"D:\Gaussian_blur_bicubic_interpolation\Classification\LR\{}\train\{}".format(i,glioma))

        for file in os.listdir(test):
            if file in os.listdir(r'D:\Gaussian_blur_bicubic_interpolation\{}_LR\t1\4x'.format(glioma)):
                shutil.copy(r'D:\Gaussian_blur_bicubic_interpolation\{}_LR\t1\4x'.format(glioma)+"\\"+file, r"D:\Gaussian_blur_bicubic_interpolation\Classification\LR\{}\test\{}".format(i,glioma))

        for file in os.listdir(val):
            if file in os.listdir(r'D:\Gaussian_blur_bicubic_interpolation\{}_LR\t1\4x'.format(glioma)):
                shutil.copy(r'D:\Gaussian_blur_bicubic_interpolation\{}_LR\t1\4x'.format(glioma)+"\\"+file, r"D:\Gaussian_blur_bicubic_interpolation\Classification\LR\{}\val\{}".format(i,glioma))